In [1]:
from iteru import *
from ipywidgets import *
ee.Initialize()

In [2]:
map = Map()

In [3]:
basemaps = Dropdown(
    options = basemaps,
    value = None,
    description = 'Basemaps',
    description_tooltip = 'Select Basemap',
    layout=Layout(width='210px'),
    indent=False
)

In [4]:
padding = "0px 0px 0px 0px" 
basemaps_button = ToggleButton(
                               value = False, 
                               tooltip = 'Select A Basemap',
                               icon = 'map',
                               layout = Layout(width = '32px', height = '28px'),
                               padding = padding
 )
TOC = ToggleButton(
                   value = False, 
                   tooltip = 'Table of Contents',
                   icon = 'align-justify',
                   layout = Layout(width = '32px', height = '28px'),
                   padding = padding
       )
basemap_tool = HBox([basemaps_button])
TOC_container = VBox([TOC])
TOC_out = Output(layout={'border': '1px solid blue'})

In [5]:
def basemap_tool_click(change):
    if change.new:
        basemap_tool.children = [basemaps_button,basemaps]
    else:
        basemap_tool.children = [basemaps_button]

def TOC_container_click(change):
    if change.new:
        TOC_container.children = [TOC,TOC_out]
    else:
        TOC_container.children = [TOC]      

TOC.observe(TOC_container_click, names = 'value')
basemaps_button.observe(basemap_tool_click, names = 'value')

basemap_tool_widget = AddWidget(widget = basemap_tool, position = 'topleft')
TOC_widget = AddWidget(widget = TOC_container, position = 'topleft')
map.add_control(basemap_tool_widget)
map.add_control(TOC_widget )

In [6]:
close_button = ToggleButton(   value = False, 
                               tooltip = 'Remove This Basemap',
                               icon = 'window-close',
                               layout = Layout(width = '32px', height = '28px'),
                               button_style="primary",)

In [7]:
def basemap_widgets(change):
    if change.new:
            if change.new in map.layers:
                pass
            else:
                map.add_layer(change.new)
                
                basemap_visibility = Checkbox(
                                              value = True, 
                                              indent=False,
                                              layout = Layout(width = '15px')
                             )
                
                basemap_name = Label(
                                   value= change.new.name,
                                   layout = Layout(width = '110px', height = '28px'),
                                   tooltip = change.new.name
                              )
                
        
                close_button = Button(  
                                      value = False, 
                                      tooltip = 'Remove This Basemap',
                                      icon = 'window-close',
                                      layout = Layout(width = '32px', height = '28px'),
                                    
                               )
            
                opacity = FloatSlider(
                                      value = 1,
                                      min = 0,  
                                      max = 1,
                                      step = 0.1,
                                      indent=False,
                                    
                               )
                
                def change_basemap_visibility(show_hide):
                    change.new.visible = show_hide 
                    
                def change_basemap_opacity(Opacity):
                    change.new.opacity = Opacity
                        
                def remove_basemap(b):
                    map.remove_layer(change.new)
                    close_button.close()
                    opacity.close()
                    basemap_name.close()
                    basemap_visibility.close()
                    
                opacity_slider = interactive(change_basemap_opacity,Opacity = opacity)
                
                visibility_checkbox = interactive(change_basemap_visibility,show_hide = basemap_visibility)
                
                with TOC_out:
                    display(HBox([basemap_visibility,basemap_name,opacity, close_button]))
                
                
                close_button.on_click(remove_basemap)
                
                           
basemaps.observe(basemap_widgets, names = 'value') 

In [8]:
map

Map(center=[27, 31], controls=(AttributionControl(options=['position', 'prefix'], position='bottomright'), Sea…

In [19]:
dem = ee.Image('CGIAR/SRTM90_V4')
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

print(dem)
print(dem.getMapId())

ee.Image({
  "functionInvocationValue": {
    "functionName": "Image.load",
    "arguments": {
      "id": {
        "constantValue": "CGIAR/SRTM90_V4"
      }
    }
  }
})
{'mapid': 'projects/earthengine-legacy/maps/586c191ed438700a520c14644f87a3fd-0f1b4f7f75be401f870151496a3b18fb', 'token': '', 'tile_fetcher': <ee.data.TileFetcher object at 0x000001C9793AFDC0>, 'image': <ee.image.Image object at 0x000001C97963ABB0>}


In [10]:
dem_id = dem.getMapId(vis_params)

In [11]:
dem_url = dem_id['tile_fetcher'].url_format

In [12]:
dem_lyr = TileLayer(
    url =dem_url, 
    attribution = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = 'DEM',
    overlay=True,
    control=True
)

In [27]:
dem2 = ee.Image('CGIAR/SRTM90_V4')


In [34]:
def add_ee_layer( ee_object, vis_params = None, name ='ee layer'):
        
        ee_object_id =ee_object.getMapId(vis_params)

        ee_object_tile = TileLayer(
        
              url = ee_object_id['tile_fetcher'].url_format,
              attribution = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
              name = name, 
              overlay = True,
              control = True
        )
        map.add_layer(ee_object_tile)

In [36]:
add_ee_layer(dem2, vis_params = vis_params, name = 'DEM3' )